## Rearranging cells according to distance from the center so we can then loop over this rearranged version and break from the loop once we find all sources

In [1]:
#Increasing the width of the notebook (visual difference only)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# starting = time.time()
# #Obtaining an iterable for the value of the midpoint of each cell
# cell_midpoints = [cell.midpoint().norm() for cell in cells(mesh)]

# #Constructing a numpy array out of the iterable, much faster instead of a list
# cell_midpoints = np.fromiter(cell_midpoints, float, mesh.num_cells())

# #Index to order all cells according to the distance of their center to the origin
# cell_sorting_index = cell_midpoints.argsort()

# #Sorting the midpoints by their radius
# cell_midpoints_sorted = cell_midpoints[cell_sorting_index]


# # mesh.cells()[0,:]
# # mesh.cells()[cell_sorting_index]
# # # mesh.cells() = mesh.cells()[cell_sorting_index]
# cells(mesh)
# # print(f'Time for midpoint: {time.time() - starting}')


#Creating a MeshFunction to indicate which cells to loop over (e.g. only the ones close to
#the point contained in the cell we will refine)
# contain_time_loop = time.time()

# contain_list.set_all(False)

# for cell in cells(mesh):
#     if cell.contains(origin):
        
#         contain_list[cell] = True

# print(f'Time for loop: {time.time()-contain_time_loop}')


In [ ]:
# mesh = mshr.generate_mesh(domain, mesh_resolution)

In [ ]:
# contain_time_list = time.time()

# index_1 = 400
# index_2 = index_1+10

# # print(f'Before. Total:{mesh.num_cells()}\n. Cells:\n {mesh.cells()[index_1:index_2]}\n')

# #Declaring Boolean Mesh Function to individuate cell containing point
# contain_function = MeshFunction("bool", mesh, 3)

# #Setting function to False everywhere
# contain_function.set_all(False)

# #setting empty array to append indices of cells containing point
# cell_index = np.zeros((source_number, 1))

# print(f'cells = {mesh.num_cells()}')

# for i, source in enumerate(source_centers):

#     #List comprehension with a single True corresponding to the cell containing the point
#     contain_list = [cell.contains(source) for cell in cells(mesh)]

#     #Converting list to a np array so it's faster (and has 0 and 1 so can be sorted)
#     contain_numpy = np.fromiter(contain_list, float, mesh.num_cells())
    
# #     print(f'contain_numpy has {contain_numpy.sum()} ones\n')
    
#     #Getting index of non-zero elements of contain_numpy, only 1 for the cell that's true.
#     #Then setting the MeshFunction True at those cells
#     cell_index[i] = np.nonzero(contain_numpy)[0]
    
# print(f'the cell index containing the point is \n {cell_index[0:10]}\n...')

# for cell_containing in cell_index:
    
#     contain_function[cell_containing] = True

# test_mesh = refine(mesh, contain_function)    
 
# print(f'cells = {test_mesh.num_cells()}')
    
# print(f'Time for list: {time.time() - contain_time_list}')


# # print(f'Before. Total:{mesh.num_cells()}\n. Cells:\n {mesh.cells()[index_1:index_2]}\n')

# contain_mesh_function = [contain_function[i] for i in range(test_mesh.num_cells())]

# contain_mesh_function_numpy = np.fromiter(contain_mesh_function, float, test_mesh.num_cells())

# contain_mesh_function_numpy.sum()

In [ ]:
def modified_refinement (mesh, location, radius, how_many, technique = 'inside'):
    '''Function to refine mesh locally, based on the distance from a given point
    '''
    
    #Starting # cells before we refine, to compute growth factor
    starting_cells = mesh.num_cells()
    
    if how_many > 0:
    
        for i in range(how_many):
        
            #Declaring Boolean Mesh Function to individuate cell containing point
            contain_function = MeshFunction("bool", mesh, 3)

            #Setting function to False everywhere
            contain_function.set_all(False)

            #setting empty array to contain indices of cells containing point
            cell_index = np.zeros((source_number, 1))
        
            for j, source in enumerate(location): 
                
                print(f'Source {j+1} of {source_number}', end="\r", flush=True)
                
                #IMPORTANT: Need to declare (and set to False) the cell_to_refine function
                #for each source, as the mesh is different after each iteration!
                #Initial mesh cell count
                initial_cells = mesh.num_cells()
                
                #List comprehension with a single True corresponding to the cell containing the point
                contain_list = [cell.contains(source) for cell in cells(mesh)]

                #Converting list to a np array so it's faster (and has 0 and 1 so can be sorted)
                contain_numpy = np.fromiter(contain_list, float, mesh.num_cells())
                
#                 print(f'contain_numpy has {contain_numpy.sum()} ones\n')

                #Getting index of non-zero elements of contain_numpy, only 1 for the cell that's true.
                #Then setting the MeshFunction True at those cells
                cell_index[j] = np.nonzero(contain_numpy)[0]

            for cell_containing in cell_index:

                contain_function[cell_containing] = True

            mesh = refine(mesh, contain_function)    
#             print(f'cell_containing is {cell_containing}')  
                
#             #Refining the mesh only where the markers are True, so inside the desired radius 
#             mesh = refine(mesh, contain_function)

            final_cells = mesh.num_cells()

            partial_growth_factor = final_cells/initial_cells
            
            print(('Iteration {} of {}: The Cell number went from {} to {}, up by a factor {}\n'
                  .format(i+1, how_many, initial_cells, final_cells, partial_growth_factor)))
    
        #ratio between # cells at beginning and end of refinement
        total_growth_factor = final_cells/starting_cells

        print('Cell number went up by a factor {}\n'.format(total_growth_factor))
    
    #returning the refined mesh
    return mesh

In [ ]:
#NOT WORKING AND CANT SEEM TO SEE WHY. MORE_MODIFIED PROBABLY FASTER ANYWAY

def adapt_modified_refinement (mesh, location, radius, how_many, technique = 'inside'):
    '''Function to refine mesh locally, based on the distance from a given point
    '''
    
    #Starting # cells before we refine, to compute growth factor
    starting_cells = mesh.num_cells()
    
    if how_many > 0:
    
        #Declaring Boolean Mesh Function to individuate cell containing point
        contain_function = MeshFunction('size_t', mesh, 3)

        #Setting function to False everywhere
        contain_function.set_all(False)

        
        for i in range(how_many):
        
            #setting empty array to contain indices of cells containing point
            cell_index = np.zeros((source_number, 1))
            
            #If not the first iteration, no need to re-define the function
            if i>0:
            
                mesh = new_mesh
            else:
                
                for j, source in enumerate(location): 

                    print(f'Source {j+1} of {source_number}', end="\r", flush=True)

                    #IMPORTANT: Need to declare (and set to False) the cell_to_refine function
                    #for each source, as the mesh is different after each iteration!
                    #Initial mesh cell count
                    initial_cells = mesh.num_cells()

                    #List comprehension with a single True corresponding to the cell containing the point
                    contain_list = [cell.contains(source) for cell in cells(mesh)]

                    #Converting list to a np array so it's faster (and has 0 and 1 so can be sorted)
                    contain_numpy = np.fromiter(contain_list, float, mesh.num_cells())

    #                 print(f'contain_numpy has {contain_numpy.sum()} ones\n')

                    #Getting index of non-zero elements of contain_numpy, only 1 for the cell that's true.
                    #Then setting the MeshFunction True at those cells
                    cell_index[j] = np.nonzero(contain_numpy)[0]

                for cell_containing in cell_index:

                    contain_function[cell_containing] = True
            
            if i==0:
            
                mesh_new = adapt(contain_function, mesh)    
#             print(f'cell_containing is {cell_containing}')  
            
            else:
            
                mesh_new  = adapt(new_contain_function, mesh)   
        
#             #Refining the mesh only where the markers are True, so inside the desired radius 
#             mesh = refine(mesh, contain_function)
            
            new_contain_function = adapt(mesh_new, contain_function)
        
            final_cells = mesh.num_cells()

            partial_growth_factor = final_cells/initial_cells
            
            print(('Iteration {} of {}: The Cell number went from {} to {}, up by a factor {}\n'
                  .format(i+1, how_many, initial_cells, final_cells, partial_growth_factor)))
    
        #ratio between # cells at beginning and end of refinement
        total_growth_factor = final_cells/starting_cells

        print('Cell number went up by a factor {}\n'.format(total_growth_factor))
    
    #returning the refined mesh
    return mesh

### Checking time difference between cell.contain and intersect. intersect works about 3 times faster and returns array with cell IDs directly!

In [ ]:
contain_time_start = time.time()
for source in source_centers:

    contain_list = [cell.contains(source) for cell in cells(mesh)]

contain_time_end = time.time()
contain_time_total = contain_time_end - contain_time_start
print(f'Contain takes {contain_time_total}s')
# print(f'Contain list is {contain_list}')

intersect_time_start = time.time()

# for source in source_centers:

intersect_list = [intersect(mesh, source).intersected_cells() for source in source_centers]

intersect_time_end = time.time()
intersect_time_total = intersect_time_end - intersect_time_start
print(f'Contain takes {intersect_time_total}s')
# print(f'Intersect list is {intersect_list}')

intersect_list[0][0]

## Trying to see if it is possible to mark the cells added after the refinement 
#### This is so we can use that information to direct the next refinement iteration (e.g. only look inside the cell that has already been refined!). This could mean that regardless of how many times we refine, after the first refinement we alsways loop over the same amount of cells! Would be a huge speed increase. One way to do this is to mark cells withina  given radius (e.g. the radius of the cell we just refined), and only look at those for refinement

In [ ]:
#IMPORTANT: Checked with simple meshes and found out that the mesh refinement does not refine
#a single cell but multiple, even though only one is selected. This means it is correct that
#there are multiple parent cells for each point (shouldnt in theory but the code is correct)

#Obtaining a list of parent cells (where the refined cell originated from) for each of the
#cells in the refined mesh. Result is a numpy array. Tried various methods from online and 
#this is the fastest (I made it, not from Stack Overflow)
parent_time_start = time.time()
parent_list = new_mesh.data().array('parent_cell', mesh.topology().dim())

#Finding the cells that are repeated (have more than one child, hence are parents), and 
#the amount of children they each have (unique_counts)
parent_unique, unique_counts = np.unique(parent_list, return_counts=True)

#Subtracting 1 from unique_counts so cells with 0 children become 0, then finding the indices
#for the values that are non-zero using np.nonzero
multi_indices = np.array(np.nonzero(unique_counts-1))

#IMPORTANT: Up to here it is all really fast and nice, but the way I assign cell_to_refine
#is really slow. However, the answer should be correct so can try test with this for now

#Making empty list to contain cells to refine
cell_to_refine = []

#Finding the index of the children cells for each parent cell in multi_indices
for element in multi_indices[0]:
    cell_to_refine.append(np.nonzero(np.where(parent_list == element, parent_list, 0)))  

cell_to_refine = np.array(cell_to_refine)
cell_to_refine = cell_to_refine.flatten()
cell_to_refine = np.concatenate(cell_to_refine).ravel()

parent_time_end = time.time()
parent_time_total = parent_time_end-parent_time_start
# print(f'Parent list takes {parent_time_total}s')
# print(f'Output = {multi_indices[0]}')
# print(f'Cells to refine #: {cell_to_refine.shape}')

In [ ]:
# start_potential_plot = time.time()
# print('Generating potential plot...\n')
# plt.figure()
# plot(u, title = 'Potential')
# # plot_format(2)
# # plt.xlabel('x')
# # plt.ylabel('y')
# plt.tight_layout()
# end_potential_plot = time.time()
# time_potential_plot_3D = run_time(end_potential_plot - start_potential_plot, '3D Potential Plot')
# section_times.append(time_potential_plot_3D)

In [ ]:
# start_acceleration_plot = time.time()
# print('Generating acceleration plot...\n')
# plt.figure()
# plot(grad(u), title = 'Acceleration')
# end_acceleration_plot = time.time()
# time_acceleration_plot_3D = run_time(end_acceleration_plot-start_acceleration_plot, '3D Acceleration Plot')
# section_times.append(time_acceleration_plot_3D)
# print('Acceleration plot generated in {}s...\n'.format(time_acceleration_plot_3D))

In [ ]:
#Little script from the same forum post to check if the integration works on a given mesh with 
#known solution. It does! If ever in doubt, check from here

# test_mesh = UnitCubeMesh(2, 4, 5)
# L = FunctionSpace(test_mesh, 'CG', 1)
# k = Expression('x[0]+x[1]+x[2]', degree=1)
# s = interpolate(k, L)

# A = np.array([0, 0, 0])
# B = np.array([1, 1, 1])

# ans = line_integral(s, A, B, n=10, show = True)
# ans0 = 3*sqrt(3)/2
# print('Error', abs(ans-ans0))

## Defining function to interpolate solution over a finer mesh, over which we can then plot and get nicer looking graphs, albeit less precise due to errors in interpolation.

In [ ]:
# #Resolution of the plane mesh
# plane_resolution = 61

# #Giving the size of the bigger square mesh
# plane_mesh_side = domain_size

# #Defining points for the corners of the mesh we need. Point locations at half the side length
# mesh_left_corner = Point(-plane_mesh_side/2, -plane_mesh_side/2)
# mesh_right_corner = Point(plane_mesh_side/2, plane_mesh_side/2)

# #Defining the mesh through a dolfin function instead of a mshr function
# plane_mesh = RectangleMesh(mesh_left_corner,mesh_right_corner,plane_resolution,plane_resolution)

# #Initialising a Mesh object (so correct dolfin mesh object we can use Mesh Functions on)
# #for the mesh made with Dolfin
# plane_mesh = Mesh(plane_mesh)

# plt.figure()
# plot(plane_mesh)

# Trying to construct mesh through MeshEditor

In [ ]:
# domain_trial = mshr.Sphere(origin, domain_size)
# mesh_trial = mshr.generate_mesh(domain_trial, 21)
# # mesh_trial

In [ ]:
# #Initialising a MeshEditor object
# mesh_edit = MeshEditor()

# #Opening an existing mesh with MeshEditor. Need to specify the cell name (string, 'tetrahedron'),
# #the topological dimension (not sure what it is!) and geometrical dimensions (3)
# mesh_edit.open(mesh_trial,mesh_trial.cell_name(), 3,3)

# #Generating three coordinate vectors to make a grid
# x_grid = np.linspace(0,1,10)
# y_grid = x_grid.copy()
# z_grid = x_grid.copy()

# #Making a 3D grid and assigning the grid values to three variables
# X, Y, Z = np.meshgrid(x_grid, y_grid, z_grid)

# #Flattening each coordinate array (ravel), then stacking them vertically, so that we now 
# #have an nd.array with 3 rows (one per coordinate, XYZ), and n columns, one per point
# #Stacked in this order to get the correct order in the array for checking correctness
# positions = np.vstack([Y.ravel(), X.ravel(), Z.ravel()])

# #Transposing the 3*n matrix into an n*3 matrix so we can use a list comprehension to assign 
# #point, as now positions[0] will give the three coordinate values instead of all n x-values!
# positions = positions.transpose()

# #List comprehension to make a list of points with coordinates from 'positions'
# grid_points = [Point(coordinates) for coordinates in positions]

# #Getting the amount of grid points
# grid_vertex_number = len(grid_points)

# #Declaring the amount of vertices in the mesh. Have to do this before adding any vertex!
# #This limits the available vertex indices, so if you try to add more than you declared, you
# #get an error!
# mesh_edit.init_vertices(grid_vertex_number)

# #Adding the vertices in grid_points to the mesh
# [mesh_edit.add_vertex(i,point) for i, point in enumerate(grid_points)]

# mesh_edit.close()

# mesh_trial.num_vertices()
# # mesh_trial.coordinates()

# #IMPORTANT: The problem with the MeshEditor is that it is not sufficient to define the vertices
# #One also has to define the cells! Need to look for a function that does that automatically
# #possibly with a Delaunay triangulation!

In [ ]:
# u_plot = interpolate(u, V_plot)
# plt.figure()
# plot(u_plot)